In [0]:
#Type de trajets
#quotidien
#occasiels
#exceptionnel



In [0]:
from pyspark.sql.functions import isnan, when, count, col,round
path = "/mnt/processed-dev/hist/Daily/CutRide/2021/12/*/"
data = spark.read.parquet(path).select("dateentry","rideid","latitude","longitude","deviceid","calculated_speed")

In [0]:
# import plotly.express as px
# #03BB6063-1C07-4450-8B5B-228D2020A051#ANONYMOUSTOKEN293
# prev_fliter = data_filtered.filter(data_filtered.rideid == "de5b8838-9cb4-4a6b-b095-722add5a911e_03BB6063-1C07-4450-8B5B-228D2020A051#ANONYMOUSTOKEN2934")
# curr_df = prev_fliter.toPandas()

# fig = px.scatter_mapbox(curr_df,lat='latitude',lon='longitude', hover_name="deviceid", mapbox_style='open-street-map',color="rideid")
# fig.update_layout(title = '', title_x=0.5)
# fig.show()

In [0]:
#  import folium
# mapp = folium.Map(location=[48.88,2.23], zoom_start=50, tiles="Stamen Terrain" )
# fg = folium.FeatureGroup(name="my map")
# lat=list(df_67["latitude"])
# lon=list(df_67["longitude"])
# for i,j in zip(lat,lon):
#   fg.add_child(folium.Marker(location=[i,j],icon=folium.Icon(color="green")))
#     #,popup=df_67.rideid
# mapp.add_child(fg)

In [0]:
def point_start_end(data):
  from pyspark.sql import Window
  import pyspark.sql.functions as F
  windows = Window.partitionBy("rideid").orderBy(F.col("dateentry").asc())

  df = (
           data.withColumn('long_end', F.last('longitude').over(windows))
            .withColumn('long_start', F.first('longitude').over(windows))
            .withColumn('lat_start', F.first('latitude').over(windows))
            .withColumn('lat_end', F.last('latitude').over(windows))

         )
  DF=df.groupby("rideid","deviceid","long_start","lat_start").agg(F.last("long_end").alias("long_end"),
                                          (F.last("lat_end").alias("lat_end")))
  return DF


  

In [0]:
DF=point_start_end(data)
  

In [0]:
def trajet_mm_sens(DF):
  tmp=DF.select('*')
  tmp2=(tmp.withColumn('lat_start2',when(tmp.lat_start > tmp.lat_end,tmp.lat_end).otherwise(tmp.lat_start))
          .withColumn('long_start2',when(tmp.lat_start > tmp.lat_end,tmp.long_end).otherwise(tmp.long_start))
          .withColumn('lat_end2',when(tmp.lat_end < tmp.lat_start,tmp.lat_start).otherwise(tmp.lat_end))
          .withColumn('long_end2',when(tmp.lat_end < tmp.lat_start,tmp.long_start).otherwise(tmp.long_end))
     )
  tmp3=tmp2.drop(tmp2.lat_start).drop(tmp2.lat_end).drop(tmp2.long_end).drop(tmp2.long_start)
  return tmp3
  

In [0]:
tmp3=trajet_mm_sens(DF)

In [0]:
tmp3.display()

rideid,deviceid,lat_start2,long_start2,lat_end2,long_end2
0007d14c-8fc4-4ea0-be7a-a8ae7b2a19b0_03BB6063-1C07-4450-8B5B-228D2020A051#ANONYMOUSTOKEN2934,03BB6063-1C07-4450-8B5B-228D2020A051#ANONYMOUSTOKEN2934,48.85598165629698,2.388899812127419,48.88400338122812,2.3673932501858985
000edb3b-0f4a-4209-8eab-7d5f93a00a5d_C1144CD1-BA67-4E0A-B2B2-107684911E2D#zzy01d10,C1144CD1-BA67-4E0A-B2B2-107684911E2D#zzy01d10,43.17034792169351,2.9892058509086272,43.187911687272226,3.0068832456843726
00476b64-175e-4ef6-92bd-daf267d552b9_7B69460F-13C1-4A97-990D-E6E2C136F41C#sxjeop10,7B69460F-13C1-4A97-990D-E6E2C136F41C#sxjeop10,45.51442690242875,4.911475563783676,45.5282662525807,4.874829622075291
0066dee4-5d80-4640-abf3-7d3723b7565b_37FAEDC5-33FC-45DE-BD82-281B25304929#67874308,37FAEDC5-33FC-45DE-BD82-281B25304929#67874308,46.22378843067272,5.208860885503278,46.274239402335255,5.089218317497633
00bae9d2-d8f0-44c5-9272-cda8218aeca7_0EEC1B45-FCBE-4438-B3A9-342599785B97#akf37z10,0EEC1B45-FCBE-4438-B3A9-342599785B97#akf37z10,43.80457131100834,4.619514388902044,43.810979020156616,4.627396204700767
00eebcc2-4b43-4dd2-b14f-db64470355af_A7BBD5B5-5260-4B73-89B1-54A791BF8FA5#n2kdcd10,A7BBD5B5-5260-4B73-89B1-54A791BF8FA5#n2kdcd10,45.052019193741764,5.085116299307257,45.11620030000001,4.9395162
00f02c55-80ce-484a-bf76-3ae83035fd46_unknown_1620334414#33553055,unknown_1620334414#33553055,48.08544,-1.6528867,48.0881151,-1.6507039
012681cb-4f95-493c-b554-654cbb93e8f7_54f78c6410482125#94834413,54f78c6410482125#94834413,44.5075527,3.2239933,44.5081932,3.218861
01345ebb-85d3-47bc-83b1-112f08bbc30d_E9C46827-FB69-4255-9B38-FB3CBF8E748E#57607766,E9C46827-FB69-4255-9B38-FB3CBF8E748E#57607766,43.51791376717716,6.9392889551882995,43.530396097380226,6.924119303009402
01549920-a774-4dd9-8577-3542f3d3c315_400CA7AC-B653-42B1-A89C-66F6C3F4AB0C#62026869,400CA7AC-B653-42B1-A89C-66F6C3F4AB0C#62026869,49.37584547264382,4.142663274121509,49.3758679930962,4.14265334732137


In [0]:
tmp_carte=tmp3.toPandas()

In [0]:
def calcule_frequence(data):
  data_2=data.groupby("deviceid",'long_start2','lat_start2','long_end2','lat_end2').count()
  df=data_2.withColumnRenamed('count', 'frequence')
  return df

In [0]:
df=calcule_frequence(tmp3)
df.display()

deviceid,long_start2,lat_start2,long_end2,lat_end2,frequence
82ABFF85-2A19-4EDF-8B29-7AB04F080A12#66668847,2.248684138136537,48.890452585080936,2.5044135935627896,49.08562111207198,1
5EE77DC2-B008-4484-B817-ADE7F558B5D7#c0e6ej10,5.228355391696694,45.61290456518143,5.224091452658646,45.61353005635516,1
2014DFDC-3786-4BF3-A5B2-09ED0DAC0DDB#89387561,0.3517820802121505,46.57012132366695,0.3518524775188281,46.57050101619928,1
475E1CC3-0FE6-4CB2-9838-0CC3C4E1C339#nhnus710,1.2781283107509098,45.81469510249755,1.2636706966620816,45.83120621225618,1
26a14b7a63ee55bd#hd3fii10,-0.4378455,43.3369818,-0.4402002,43.3370357,1
7B69460F-13C1-4A97-990D-E6E2C136F41C#sxjeop10,4.684863357892279,45.362855561812886,4.869572587743391,45.54011029445025,1
108fc6b692e9397a#42315007,2.2549333,48.7058452,2.2216474,48.7067255,1
3E513797-AC31-4BDF-9B82-26D58010368C#17896153,2.3699121367401688,48.82554778715746,2.368188884253727,48.827365978725936,1
5EE77DC2-B008-4484-B817-ADE7F558B5D7#c0e6ej10,5.09781068574182,45.5976917198288,5.132169310329747,45.64306784185567,1
78950B9C-3E7B-4C0E-B492-D2849D352617#16363890,2.310840351169186,48.84522464310563,2.290987077865573,48.86219205984514,1


In [0]:
# initialement 4  qui bouge pas 
tmp3.where((tmp3.lat_start2 == tmp3.lat_end2) & (tmp3.long_start2 == tmp3.long_end2)).count()

Out[9]: 4

In [0]:
from math import radians, cos, sin, asin, sqrt
@udf("double")
def get_distance(longit_a, latit_a, longit_b, latit_b):
  if longit_a is None or latit_a is None or longit_b is None or latit_b is None:
    return 0
  # Transform to radians
  longit_a, latit_a, longit_b, latit_b = map(radians, [longit_a, latit_a, longit_b, latit_b])
  dist_longit = longit_b - longit_a
  dist_latit = latit_b - latit_a
  # Calculate area
  area = sin(dist_latit / 2) ** 2 + cos(latit_a) * cos(latit_b) * sin(dist_longit / 2) ** 2
  # Calculate the central angle
  central_angle = 2 * asin(sqrt(area))
  radius = 6371000
  # Calculate Distance in meter
  distance = central_angle * radius
  return abs(distance)

In [0]:
from pyspark.sql import Window
w = Window().partitionBy("rideid").orderBy(col("deviceid").asc(), col("dateentry").asc())
data_2 = data
data_2 = data_2.select("*", F.lag("dateentry").over(w).alias("previousDateentry")).na.drop()
data_2 = data_2.select("*", F.lag("deviceid").over(w).alias("previousDeviceid")).na.drop()
data_2 = data_2.select("*", F.lag("latitude").over(w).alias("previousLatitude")).na.drop()
data_2 = data_2.select("*", F.lag("longitude").over(w).alias("previousLongitude")).na.drop()
data_2 = data_2.select("*", F.lag("calculated_speed").over(w).alias("previousSpeed")).na.drop()

In [0]:
data_2 = data_2.withColumn("distance",get_distance(data_2.longitude, data_2.latitude, data_2.previousLongitude, data_2.previousLatitude))

In [0]:
data_2 = data_2.withColumn("distance_km",F.col("distance")/1000)

In [0]:
data_2.groupBy("rideid").agg(F.sum("distance_km").alias("distance_trajet")).display()

rideid,distance_trajet
000edb3b-0f4a-4209-8eab-7d5f93a00a5d_C1144CD1-BA67-4E0A-B2B2-107684911E2D#zzy01d10,11.447947581657742
0cb47289-0e5f-4df7-a720-9c51beb418d2_8DFDC300-AEFF-4647-9258-A6EB77568B41#21923615,72.81760287623318
0dcdc777-012f-4899-9bb0-d5a0cb16db33_B4C62706-DD0D-4B18-A12D-19327906FBC4#ANONYMOUSTOKEN3232,15.252612759633537
133c20a8-2f7f-461b-8936-b9ea7e56a7f3_ac437a9cb2c39dcd#ANONYMOUSTOKEN4080,5.27471338039335
18c01aeb-756b-4302-9682-d92a0692f587_9CDF1105-373E-4256-9499-92274051AAD9#78786142,12.612629300288054
19390789-299b-4c5f-8f0c-169ad19ea601_83b62548047ef262#akinot10,7.578016074655895
36024c3e-4f8a-46f6-bfcf-25c8ea6a4db4_B1D6E639-F27E-42F3-B7E6-CC3CC407A35D#88333950,3.768008599634161
4e75f869-2ad5-4c74-8e0a-fef58a9a625d_5EE77DC2-B008-4484-B817-ADE7F558B5D7#c0e6ej10,18.12405215842721
56e769ab-2f45-4d25-9158-05e5410bf72b_400CA7AC-B653-42B1-A89C-66F6C3F4AB0C#62026869,9.534845355129466
5769f039-6416-4c67-9301-f9eb81e15462_F1274677-9E9E-4065-B873-A39049DAB83B#48140595,22.790877973951435


In [0]:
def  func_round(data,a):
  from pyspark.sql.functions import isnan, when, count, col,round
  import pyspark.sql.functions as F
  tmp=data.select('rideid','deviceid',(round('long_start2',a)).alias('long_start'),(round('lat_start2',a)).alias('lat_start'),
                (round('long_end2',a)).alias('long_end'),(round('lat_end2',a)).alias('lat_end'))
  tmp_total=tmp.withColumn('distance',distance(tmp.lat_start,tmp.long_start,tmp.lat_end,tmp.long_end))
  tmp_distance=tmp_total.where(tmp_total.distance != 0)
  tmp_distance_0=tmp_total.where(tmp_total.distance == 0)
  tmp_total,tmp_distance,tmp_distance_0 
  return  tmp_total,tmp_distance,tmp_distance_0 
  
  

In [0]:
  import pyspark.sql.functions as F
  tmp_total,tmp_distance,tmp_distance_0 = func_round(tmp3,2)

In [0]:
0007d14c-8fc4-4ea0-be7a-a8ae7b2a19b0_03BB6063-1C07-4450-8B5B-228D2020A051#ANONYMOUSTOKEN2934

In [0]:
tmp_total.display()

rideid,deviceid,long_start,lat_start,long_end,lat_end,distance
0007d14c-8fc4-4ea0-be7a-a8ae7b2a19b0_03BB6063-1C07-4450-8B5B-228D2020A051#ANONYMOUSTOKEN2934,03BB6063-1C07-4450-8B5B-228D2020A051#ANONYMOUSTOKEN2934,2.39,48.86,2.37,48.88,1.6540087591893433
000edb3b-0f4a-4209-8eab-7d5f93a00a5d_C1144CD1-BA67-4E0A-B2B2-107684911E2D#zzy01d10,C1144CD1-BA67-4E0A-B2B2-107684911E2D#zzy01d10,2.99,43.17,3.01,43.19,1.7102481403496357
00476b64-175e-4ef6-92bd-daf267d552b9_7B69460F-13C1-4A97-990D-E6E2C136F41C#sxjeop10,7B69460F-13C1-4A97-990D-E6E2C136F41C#sxjeop10,4.91,45.51,4.87,45.53,2.3789380737575594
0066dee4-5d80-4640-abf3-7d3723b7565b_37FAEDC5-33FC-45DE-BD82-281B25304929#67874308,37FAEDC5-33FC-45DE-BD82-281B25304929#67874308,5.21,46.22,5.09,46.27,6.694281316021081
00bae9d2-d8f0-44c5-9272-cda8218aeca7_0EEC1B45-FCBE-4438-B3A9-342599785B97#akf37z10,0EEC1B45-FCBE-4438-B3A9-342599785B97#akf37z10,4.62,43.8,4.63,43.81,0.8520780411939922
00eebcc2-4b43-4dd2-b14f-db64470355af_A7BBD5B5-5260-4B73-89B1-54A791BF8FA5#n2kdcd10,A7BBD5B5-5260-4B73-89B1-54A791BF8FA5#n2kdcd10,5.09,45.05,4.94,45.12,8.771481780217625
00f02c55-80ce-484a-bf76-3ae83035fd46_unknown_1620334414#33553055,unknown_1620334414#33553055,-1.65,48.09,-1.65,48.09,0.0
012681cb-4f95-493c-b554-654cbb93e8f7_54f78c6410482125#94834413,54f78c6410482125#94834413,3.22,44.51,3.22,44.51,0.0
01345ebb-85d3-47bc-83b1-112f08bbc30d_E9C46827-FB69-4255-9B38-FB3CBF8E748E#57607766,E9C46827-FB69-4255-9B38-FB3CBF8E748E#57607766,6.94,43.52,6.92,43.53,1.217088069549791
01549920-a774-4dd9-8577-3542f3d3c315_400CA7AC-B653-42B1-A89C-66F6C3F4AB0C#62026869,400CA7AC-B653-42B1-A89C-66F6C3F4AB0C#62026869,4.14,49.38,4.14,49.38,0.0


In [0]:
print(tmp_total.count())
print(tmp_distance_0.count())
print(tmp_distance.count())

4379
763
3616

In [0]:
# avec un Km pres il y a 763 ( 17%)
print((tmp_distance_0.count()/tmp_total.count())*100)

17.42406942224252

In [0]:
df_carte=tmp_distance_0.toPandas()

In [0]:
import plotly.express as px
fig = px.scatter_mapbox(df_carte,lat='lat_start',lon='long_start', mapbox_style='open-street-map')
fig.update_layout(title = 'map', title_x=0.5)
fig.show()

In [0]:
def calcule_frequence(data):
  data_2=data.groupby("deviceid",'long_start','lat_start','long_end','lat_end').count()
  df=data_2.withColumnRenamed('count', 'frequence')
  return df

 

  

In [0]:
df=calcule_frequence(tmp_total)
df.select(F.max("frequence")).show()

+--------------+
max(frequence)|
+--------------+
 67|
+--------------+

In [0]:
var = 1

In [0]:
df.count()

Out[17]: 2550

In [0]:
df.filter(df.frequence == 67).display()

deviceid,long_start,lat_start,long_end,lat_end,frequence
03BB6063-1C07-4450-8B5B-228D2020A051#ANONYMOUSTOKEN2934,2.37,48.88,2.37,48.88,67


In [0]:
import folium
map_osm = folium.Map(location=[48.88, 2.37])
map_osm.add_child(folium.RegularPolygonMarker(location=[48.88, 2.37], fill_color='#132b5e', radius=5))
map_osm

Out[36]:

Make this Notebook Trusted to load map: File -> Trust Notebook <iframe src="about:blank" style="position:absolute;width:100%;height:100%;left:0;top:0;border:none !important;" data-html=%3C%21DOCTYPE%20html%3E%0A%3Chead%3E%20%20%20%20%0A%20%20%20%20%3Cmeta%20http-equiv%3D%22content-type%22%20content%3D%22text/html%3B%20charset%3DUTF-8%22%20/%3E%0A%20%20%20%20%0A%20%20%20%20%20%20%20%20%3Cscript%3E%0A%20%20%20%20%20%20%20%20%20%20%20%20L_NO_TOUCH%20%3D%20false%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20L_DISABLE_3D%20%3D%20false%3B%0A%20%20%20%20%20%20%20%20%3C/script%3E%0A%20%20%20%20%0A%20%20%20%20%3Cstyle%3Ehtml%2C%20body%20%7Bwidth%3A%20100%25%3Bheight%3A%20100%25%3Bmargin%3A%200%3Bpadding%3A%200%3B%7D%3C/style%3E%0A%20%20%20%20%3Cstyle%3E%23map%20%7Bposition%3Aabsolute%3Btop%3A0%3Bbottom%3A0%3Bright%3A0%3Bleft%3A0%3B%7D%3C/style%3E%0A%20%20%20%20%3Cscript%20src%3D%22https%3A//cdn.jsdelivr.net/npm/leaflet%401.6.0/dist/leaflet.js%22%3E%3C/script%3E%0A%20%20%20%20%3Cscript%20src%3D%22https%3A//code.jquery.com/jquery-1.12.4.min.js%22%3E%3C/script%3E%0A%20%20%20%20%3Cscript%20src%3D%22https%3A//maxcdn.bootstrapcdn.com/bootstrap/3.2.0/js/bootstrap.min.js%22%3E%3C/script%3E%0A%20%20%20%20%3Cscript%20src%3D%22https%3A//cdnjs.cloudflare.com/ajax/libs/Leaflet.awesome-markers/2.0.2/leaflet.awesome-markers.js%22%3E%3C/script%3E%0A%20%20%20%20%3Clink%20rel%3D%22stylesheet%22%20href%3D%22https%3A//cdn.jsdelivr.net/npm/leaflet%401.6.0/dist/leaflet.css%22/%3E%0A%20%20%20%20%3Clink%20rel%3D%22stylesheet%22%20href%3D%22https%3A//maxcdn.bootstrapcdn.com/bootstrap/3.2.0/css/bootstrap.min.css%22/%3E%0A%20%20%20%20%3Clink%20rel%3D%22stylesheet%22%20href%3D%22https%3A//maxcdn.bootstrapcdn.com/bootstrap/3.2.0/css/bootstrap-theme.min.css%22/%3E%0A%20%20%20%20%3Clink%20rel%3D%22stylesheet%22%20href%3D%22https%3A//maxcdn.bootstrapcdn.com/font-awesome/4.6.3/css/font-awesome.min.css%22/%3E%0A%20%20%20%20%3Clink%20rel%3D%22stylesheet%22%20href%3D%22https%3A//cdnjs.cloudflare.com/ajax/libs/Leaflet.awesome-markers/2.0.2/leaflet.awesome-markers.css%22/%3E%0A%20%20%20%20%3Clink%20rel%3D%22stylesheet%22%20href%3D%22https%3A//cdn.jsdelivr.net/gh/python-visualization/folium/folium/templates/leaflet.awesome.rotate.min.css%22/%3E%0A%20%20%20%20%0A%20%20%20%20%20%20%20%20%20%20%20%20%3Cmeta%20name%3D%22viewport%22%20content%3D%22width%3Ddevice-width%2C%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20initial-scale%3D1.0%2C%20maximum-scale%3D1.0%2C%20user-scalable%3Dno%22%20/%3E%0A%20%20%20%20%20%20%20%20%20%20%20%20%3Cstyle%3E%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%23map_56ca387cc40c46dc9aaea783540fcb12%20%7B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20position%3A%20relative%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20width%3A%20100.0%25%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20height%3A%20100.0%25%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20left%3A%200.0%25%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20top%3A%200.0%25%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%7D%0A%20%20%20%20%20%20%20%20%20%20%20%20%3C/style%3E%0A%20%20%20%20%20%20%20%20%0A%20%20%20%20%3Cscript%20src%3D%22https%3A//cdnjs.cloudflare.com/ajax/libs/leaflet-dvf/0.3.0/leaflet-dvf.markers.min.js%22%3E%3C/script%3E%0A%3C/head%3E%0A%3Cbody%3E%20%20%20%20%0A%20%20%20%20%0A%20%20%20%20%20%20%20%20%20%20%20%20%3Cdiv%20class%3D%22folium-map%22%20id%3D%22map_56ca387cc40c46dc9aaea783540fcb12%22%20%3E%3C/div%3E%0A%20%20%20%20%20%20%20%20%0A%3C/body%3E%0A%3Cscript%3E%20%20%20%20%0A%20%20%20%20%0A%20%20%20%20%20%20%20%20%20%20%20%20var%20map_56ca387cc40c46dc9aaea783540fcb12%20%3D%20L.map%28%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%22map_56ca387cc40c46dc9aaea783540fcb12%22%2C%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%7B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20center%3A%20%5B48.88%2C%202.37%5D%2C%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20

In [0]:
def frequence_percent(data):
  from pyspark.sql import Window
  data_1=data.groupBy('frequence').count().orderBy('count', ascending=False)
  data_2=data_1.withColumn('percent', F.col('count')/F.sum('count').over(Window.partitionBy())*100)
  return data_2


In [0]:
df2=frequence_percent(df)

frequence,count,percent
1,1834,79.15407854984893
2,268,11.566681053085887
3,73,3.150625809236081
4,53,2.2874406560207166
5,26,1.122140699179974
7,16,0.6905481225722918
6,12,0.5179110919292188
8,6,0.2589555459646094
9,4,0.17263703064307295
18,4,0.17263703064307295


In [0]:
#Type de trajets
#quotidien
#occasiels
#exceptionnel




###reste à faire
#""#data_21.select(data_21.frequence, F.when(data_21.frequence> X1, "quotidien").when(data_21.frequence < X2, "exceptionnel").otherwise("occasiels")).show()